In [13]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import phasepy as pp
import CoolProp.CoolProp as CP
import pandas as pd

In [14]:
# Definición de funciones
def choose(c1, c2, c3, c4, pick):
    return [c1, c2, c3, c4][pick - 1]

def adiW(t, T1, Cv):
    return Cv * (t - T1) / 1000

def plot_expansion_compression(bn, pick1, pick2, go, Pf):
    # Crear objeto nitrógeno, datos obtenidos del Perry's Handbook
    nitrogen = pp.component(name = 'nitrogen', Tc = 162.2, Pc = 3.39e2, Zc = 0.288, Vc = 89, w = 0.037)
    
    # Modelado de compuestos con Peng-Robinson
    eos_nitrogeno = pp.preos(nitrogen)
    
    # Cálculo
    P1 = 0.1 * 10**6 if bn == 1 else 1 * 10**6
    P2 = Pf * 10**6

    R = 8.314
    T1 = 300
    V1 = 1 / eos_nitrogeno.density(T1, P1*1e-5, state='V') / 1000

    # Isotérmico
    isoV2 = 1 / eos_nitrogeno.density(T1, P2*1e-5, state='V') / 1000
    isoWR = -R * T1 * np.log(isoV2 / V1) / 1000
    isoWX = -P2 * (isoV2 - V1) / 1000 / 1000

    # Adiabático 
    Cp = CP.PropsSI('CPMOLAR', 'T', T1, 'P', P1, 'Nitrogen') 
    Cv = CP.PropsSI('CVMOLAR', 'T', T1, 'P', P1, 'Nitrogen') 
    gamma = Cp / Cv 

    """ Se podría utilizar un promedio entre Cp y Cv, pero esto complica mucho los cálculos y no es necesario pues Cp y Cv varían muy poco con T """
    
    adiV2R = V1 * (P1 / P2)**(1 / gamma)
    adiV2X = 1000 * (R * (Cv * T1 + P2 * V1 / 1000)) / (P2 * (Cv + R))
    T2R = T1 * (V1 / adiV2R)**(gamma - 1)
    T2X = T1 - P2 * (adiV2X - V1) / 1000 / Cv

    v1 = V1 + (choose(isoV2, adiV2R, isoV2, adiV2X, pick1) - V1) * go
    v2 = V1 + (choose(isoV2, adiV2R, isoV2, adiV2X, pick2) - V1) * go

   # Presiones intermedias
    Pint = (P1 + (P2 - P1) * go) / 10**6
    Pext1 = Pint if pick1 in [1, 2] else P2 / 10**6
    Pext2 = Pint if pick2 in [1, 2] else P2 / 10**6

    # Cálculos de trabajo
    W1 = choose(isoWR, adiW(T2R, T1, Cv), isoWX, adiW(T2X, T1, Cv), pick1) * go
    T1_final = T1 + (choose(T1, T2R, T1, T2X, pick1) - T1) * go

    W2 = choose(isoWR, adiW(T2R, T1, Cv), isoWX, adiW(T2X, T1, Cv), pick2) * go
    T2_final = T1 + (choose(T1, T2R, T1, T2X, pick2) - T1) * go

    # Datos para la tabla
    data = {
        'Condición': ['Condición 1', 'Condición 2'],
        'Proceso': [choose('Reversible Isotérmico', 'Reversible Adiabático', 'Irreversible Isotérmico', 'Irreversible Adiabático', pick1),
                    choose('Reversible Isotérmico', 'Reversible Adiabático', 'Irreversible Isotérmico', 'Irreversible Adiabático', pick2)],
        'Trabajo (kJ/mol)': [W1, W2],
        'Temperatura (K)': [T1_final, T2_final],
        'Volumen (L)': [v1, v2],
        'Presión (MPa)': [Pext1, Pext2]
    }

    df = pd.DataFrame(data)

    # Graficación
    fig, ax = plt.subplots()
    delta_x, delta_w, delta_h, th, height = 10, 2, 1, 0.5, 1

    # Rectángulos que representan los volúmenes
    rect1 = plt.Rectangle((0, 0), delta_x, v1, edgecolor='black', facecolor='green', alpha=0.7)
    rect2 = plt.Rectangle((delta_x + delta_w, 0), delta_x, v2, edgecolor='black', facecolor='green', alpha=0.7)
    rect3 = plt.Rectangle((0, v1), delta_x, 1.5, edgecolor='black', facecolor='gray')
    rect4 = plt.Rectangle((delta_x + delta_w, v2), delta_x, 1.5, edgecolor='black', facecolor='gray')


    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(rect3)
    ax.add_patch(rect4)

    # Configuración de los límites del gráfico
    plt.xlim(-1, 2 * delta_x + delta_w + 1)
    plt.ylim(-1, max(v1, v2) + 2.5)

    # Etiquetas de texto para presiones
    ax.text(delta_x / 2, 0.5 * v1, f"P = {Pint:.2f} MPa", fontsize=12, ha='center')
    ax.text(1.5 * delta_x + delta_w, 0.5 * v2, f"P = {Pint:.2f} MPa", fontsize=12, ha='center')
    ax.text(delta_x / 2, v1 + 1, f"Pext = {Pext1:.2f} MPa", fontsize=12, ha='center')
    ax.text(1.5 * delta_x + delta_w, v2 + 1, f"Pext = {Pext2:.2f} MPa", fontsize=12, ha='center')

    # Títulos y etiquetas del gráfico
    plt.title("Reversible and Irreversible Expansion or Compression Work")
    plt.grid(False)
    plt.axis('off')
    plt.show()

    # Mostrar la tabla
    display(df)

# Widgets interactivos para la selección de parámetros
interact(plot_expansion_compression,
         bn=widgets.ToggleButtons(options=[('Compression', 1)], description='Process:'),
         pick1=widgets.Dropdown(options=[('Reversible Isothermal', 1), ('Reversible Adiabatic', 2), ('Irreversible Isothermal', 3), ('Irreversible Adiabatic', 4)], description='Condition 1:'),
         pick2=widgets.Dropdown(options=[('Reversible Isothermal', 1), ('Reversible Adiabatic', 2), ('Irreversible Isothermal', 3), ('Irreversible Adiabatic', 4)], description='Condition 2:'),
         go=widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='Progress:'),
         Pf=widgets.FloatSlider(value=1.5, min=0.1, max=2.0, step=0.1, description='Final Pressure (MPa):'))

interactive(children=(ToggleButtons(description='Process:', options=(('Compression', 1),), value=1), Dropdown(…

<function __main__.plot_expansion_compression(bn, pick1, pick2, go, Pf)>